In [148]:
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.corpus import webtext
from autocorrect import Speller

#### ***Process to Discovering Word Collocations***

In [63]:
## converting to lowercase and putting them into dictionary
#words = [w.lower() for w in webtext.words('grail.txt')]
words = [w.lower() for w in webtext.words('singles.txt')]
output = ' '.join(words)
print(output)
print('Total Number of Words :', len(words))
print('Total Number of Characters :',len(output))

25 sexy male , seeks attrac older single lady , for discreet encounters . 35yo security guard , seeking lady in uniform for fun times . 40 yo single dad , sincere friendly dte seeks r / ship with fem age open s / e 44yo tall seeks working single mum or lady below 45 fship rship . nat open 6 . 2 35 yr old outgoing m seeks fem 28 - 35 for o / door sports - w / e away a professional business male , late 40s , 6 feet tall , slim build , well groomed , great personality , home owner , interests include the arts travel and all things good , ringwood area , is seeking a genuine female of similar age or older , in same area or surrounds , for a meaningful long term rship . looking forward to hearing from you all . able young man seeks , sexy older women . phone for fun ready to play affectionate lady sought by generous guy , 40s , mutual fulfillment are you alone or lost in a r / ship too , with no hope in sight ? maybe we could explore new beginnings together ? im 45 slim / med build , gsoh ,

In [64]:
## Removing puncutions from string
removedpunctuationsList = re.sub(r'[^\w\s]', '', output)
print(removedpunctuationsList)
print(len(removedpunctuationsList))
newlist = []
for i in words:
    if re.match(r'[^\w\s]', i):
        continue
    else:
        newlist.append(i)
print(len(newlist))
print(len(words))

25 sexy male  seeks attrac older single lady  for discreet encounters  35yo security guard  seeking lady in uniform for fun times  40 yo single dad  sincere friendly dte seeks r  ship with fem age open s  e 44yo tall seeks working single mum or lady below 45 fship rship  nat open 6  2 35 yr old outgoing m seeks fem 28  35 for o  door sports  w  e away a professional business male  late 40s  6 feet tall  slim build  well groomed  great personality  home owner  interests include the arts travel and all things good  ringwood area  is seeking a genuine female of similar age or older  in same area or surrounds  for a meaningful long term rship  looking forward to hearing from you all  able young man seeks  sexy older women  phone for fun ready to play affectionate lady sought by generous guy  40s  mutual fulfillment are you alone or lost in a r  ship too  with no hope in sight  maybe we could explore new beginnings together  im 45 slim  med build  gsoh  high needs and looking for someone si

In [65]:
## removing stopwords
wordList = stopwords.words('english')
tokenzied_list = word_tokenize(removedpunctuationsList)
list_without_spotwords = []
for x in tokenzied_list:
    if x not in wordList:
        list_without_spotwords.append(x)

final_string = ' '.join(list_without_spotwords)
print(final_string)

25 sexy male seeks attrac older single lady discreet encounters 35yo security guard seeking lady uniform fun times 40 yo single dad sincere friendly dte seeks r ship fem age open e 44yo tall seeks working single mum lady 45 fship rship nat open 6 2 35 yr old outgoing seeks fem 28 35 door sports w e away professional business male late 40s 6 feet tall slim build well groomed great personality home owner interests include arts travel things good ringwood area seeking genuine female similar age older area surrounds meaningful long term rship looking forward hearing able young man seeks sexy older women phone fun ready play affectionate lady sought generous guy 40s mutual fulfillment alone lost r ship hope sight maybe could explore new beginnings together im 45 slim med build gsoh high needs looking someone similar wont disappointed amiable 43 gentleman european background 170 cm medium build employed never married children enjoys sports music cafes beach c seeks honest attractive lady eur

In [66]:
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures


bcf = BigramCollocationFinder.from_words(words)

print(bcf.nbest(BigramAssocMeasures.likelihood_ratio,10))  # only print out 10 combination

[('/', 'ship'), ('looking', 'for'), ('would', 'like'), ('.', 'o'), ('r', '/'), ('y', '.'), ('like', 'to'), ('to', 'meet'), ('5', 'ft'), ('age', 'open')]


In [76]:
stopset = set(stopwords.words('english'))

filter_stop = lambda w: len(w)<3 or w in stopset

bcf.apply_word_filter(filter_stop)
n = 2
bcf.apply_freq_filter(n)
print(bcf.nbest(BigramAssocMeasures.likelihood_ratio,10))

[('would', 'like'), ('age', 'open'), ('medium', 'build'), ('social', 'drinker'), ('non', 'smoker'), ('quiet', 'nights'), ('long', 'term'), ('easy', 'going'), ('poss', 'rship'), ('fun', 'times')]


In [77]:
tcf = TrigramCollocationFinder.from_words(words)
tcf.apply_word_filter(filter_stop)
tcf.apply_freq_filter(n)
print(tcf.nbest(TrigramAssocMeasures.likelihood_ratio, 10))


[('lady', 'age', 'open'), ('long', 'term', 'relationship'), ('guy', 'late', '30s'), ('seeks', 'married', 'lady'), ('fun', 'loving', 'lady'), ('married', 'male', 'seeks')]


In [69]:
print(len(re.findall('clop clop clop', output)))

0


In [78]:
for k,v in tcf.ngram_fd.items():
    print(k,v)

('guy', 'late', '30s') 2
('fun', 'loving', 'lady') 2
('long', 'term', 'relationship') 3
('lady', 'age', 'open') 2
('married', 'male', 'seeks') 2
('seeks', 'married', 'lady') 2


In [79]:
for k,v in bcf.ngram_fd.items():
    print(k,v)

('fun', 'times') 7
('age', 'open') 12
('slim', 'build') 6
('long', 'term') 6
('medium', 'build') 10
('attractive', 'lady') 5
('would', 'like') 13
('quiet', 'nights') 7
('non', 'smoker') 11
('similar', 'interests') 5
('good', 'looking') 6
('social', 'drinker') 9
('seeks', 'lady') 14
('easy', 'going') 5
('poss', 'rship') 7
('male', 'seeks') 6


#### ***Pointwise mutual information*** 

In [72]:
from nltk.collections import *
text = "this is a foo bar bar black sheep foo bar bar black sheep shep bar bar black sentence"
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(word_tokenize(text))
for i in finder.score_ngrams(trigram_measures.pmi):
    print(i)


(('this', 'is', 'a'), 8.339850002884624)
(('is', 'a', 'foo'), 7.339850002884624)
(('black', 'sheep', 'shep'), 5.754887502163468)
(('a', 'foo', 'bar'), 4.754887502163468)
(('black', 'sheep', 'foo'), 4.754887502163468)
(('sheep', 'shep', 'bar'), 4.754887502163468)
(('bar', 'black', 'sentence'), 4.169925001442312)
(('bar', 'black', 'sheep'), 4.169925001442312)
(('sheep', 'foo', 'bar'), 3.7548875021634682)
(('bar', 'bar', 'black'), 3.169925001442312)
(('foo', 'bar', 'bar'), 3.169925001442312)
(('shep', 'bar', 'bar'), 3.169925001442312)


In [73]:
from nltk.collections import *
text = "this is a foo bar bar black sheep foo bar bar black sheep shep bar bar black sentence"
biigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(word_tokenize(text))
for i in finder.score_ngrams(biigram_measures.pmi):
    print(i)


(('is', 'a'), 4.169925001442312)
(('this', 'is'), 4.169925001442312)
(('a', 'foo'), 3.169925001442312)
(('sheep', 'shep'), 3.169925001442312)
(('black', 'sentence'), 2.584962500721156)
(('black', 'sheep'), 2.584962500721156)
(('sheep', 'foo'), 2.169925001442312)
(('bar', 'black'), 1.584962500721156)
(('foo', 'bar'), 1.584962500721156)
(('shep', 'bar'), 1.584962500721156)
(('bar', 'bar'), 0.5849625007211561)


#### ***Synonym and antonym***

In [85]:
syn = wordnet.synsets('spring')
print(syn)
print(len(syn))

[Synset('spring.n.01'), Synset('spring.n.02'), Synset('spring.n.03'), Synset('spring.n.04'), Synset('give.n.01'), Synset('leap.n.01'), Synset('jump.v.01'), Synset('form.v.03'), Synset('bounce.v.01'), Synset('spring.v.04'), Synset('spring.v.05')]
11


In [95]:
sysn = []
antn = []

input_word = input()

for syn in wordnet.synsets(input_word):
    for l in syn.lemmas():
        sysn.append(l.name())
        if l.antonyms():
            antn.append(l.antonyms()[0].name())
        
print(set(sysn))
print("------------------------------------")
print(set(antn))

{'exam', 'tryout', 'screen', 'test', 'psychometric_test', 'quiz', 'try_out', 'essay', 'trial', 'mental_testing', 'trial_run', 'prove', 'try', 'run', 'examine', 'examination', 'mental_test'}
------------------------------------
set()


In [129]:
syns = wordnet.synsets('jump')
print('sysns : ', syns)

print('Name: ', syns[3].name())

print('Lemma name:', syns[10].lemmas()[0].name())

print('Defination of the word : ', syns[0].definition())

print('Example of word :', syns[0].examples())

sysns :  [Synset('jump.n.01'), Synset('leap.n.02'), Synset('jump.n.03'), Synset('startle.n.01'), Synset('jump.n.05'), Synset('jump.n.06'), Synset('jump.v.01'), Synset('startle.v.02'), Synset('jump.v.03'), Synset('jump.v.04'), Synset('leap_out.v.01'), Synset('jump.v.06'), Synset('rise.v.11'), Synset('jump.v.08'), Synset('derail.v.02'), Synset('chute.v.01'), Synset('jump.v.11'), Synset('jumpstart.v.01'), Synset('jump.v.13'), Synset('leap.v.02'), Synset('alternate.v.01')]
Name:  startle.n.01
Lemma name: leap_out
Defination of the word :  a sudden and decisive increase
Example of word : ['a jump in attendance']


#### ***Simmilarity b/w words***

In [1]:
w1 = wordnet.synset('ship.n.01')
w2 = wordnet.synset('boat.n.01')

print(w1.wup_similarity(w2))


NameError: name 'wordnet' is not defined

#### ***Fixing Word Lengthening***

In [144]:
text = 'helooo'

patt = re.compile(r"(.)\1{2,}")
print(patt.sub(r"\1\1",text))


heloo


In [147]:
str1 = "Emma's luck number are 251 761 231 451"

string_pattern = r"\d{3}"

regex_pattern = re.compile(string_pattern)

print("Type of pattern: ", type(regex_pattern))

result = regex_pattern.findall(str1)

print('Output from str1 :', result)

str2 = "Kelly's 33 luck numbers are 111 212 415 12124"

result = regex_pattern.findall(str2)
print("Output from str2:", result)

Type of pattern:  <class 're.Pattern'>
Output from str1 : ['251', '761', '231', '451']
Output from str2: ['111', '212', '415', '121']


In [156]:
text = 'helooo'

patt = re.compile(r"(.)\1{2,}")
print(patt.sub(r"\1\1",text))


heloo


#### ***Spell Checker Autocorrect***

In [159]:
spell = Speller(lang='en')
print(spell('Mussage'))
print(spell('helooo'))

Message
hello
